### Module Imports

In [1]:
# Imports:
import pandas as pd
import numpy as np

### Data Imports

The original data was found on [Kaggle](https://www.kaggle.com/datasets/mikhailma/energy-efficiency-of-buildings-in-new-york) as a practice dataset, and is what inspired me to do this project.  It contains 40 columns, one of which is an 'Energy Star Score' and the others are features related to location, property type, size, and energy and water use.

This dataset does cover three years 2015-2017, which is why it seems to have a lot more entries. But I think it wasn't combined very well and that's part of why there are so many missing values.

After running my initial models with this data, I wasn't happy with my results and looked online for additional data.  I went ahead and used the new data for my modeling, but I am keeping this here as a reference.

In [2]:
# initialize original dataframe that i can refer back to if needed
buildings = pd.read_csv('./data/NYC_Energy_Water_Data_2015_2017.csv', low_memory=False, index_col=0)
buildings.head()

,Property Name,Parent Property Id,Parent Property Name,Street Number,Street Name,Borough,Primary Property Type - Self Selected,List of All Property Use Types at Property,Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),...,District Steam Use (kBtu),Natural Gas Use (kBtu),Weather Normalized Site Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Weather Normalized Site Electricity (kWh),Total GHG Emissions (Metric Tons CO2e),Direct GHG Emissions (Metric Tons CO2e),Indirect GHG Emissions (Metric Tons CO2e),Water Use (All Water Sources) (kgal),DOF Benchmarking Submission Status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,NaN,NaN,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,...,1133475.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,3635.5,Not found
3,Cathedral Preparatory Seminary,NaN,NaN,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,...,NaN,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,102.9,Not found
4,The Nomad Hotel,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,...,NaN,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,10762.6,Not found
5,10 West 27 Street Corp,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,790.1,Not found
6,Westbury Realty,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,143.0,Not found


Here is the additional data I found from [opendata.cityofnewyork.us](https://data.cityofnewyork.us/browse?q=energy%20and%20water%20disclosure&sortBy=relevance).  I found that the increased number of features helps my model, so I went ahead and worked with these.

I selected four years of data (2018-2021) because each year's datasets had a similar number of features, and so I decided to start with the newest one and add more as needed.  I kept adding datasets to my `combined` dataframe below until I had over 40K entries, which was 4 datasets.  I could definitely add more, but I would just end up with longer runtime for my models without many other improvements. 

In [3]:
b2021 = pd.read_csv('./data/Energy_and_Water_Data_2021.csv', low_memory=False, index_col=0)

In [4]:
b2020 = pd.read_csv('./data/Energy_and_Water_Data_2020.csv', low_memory=False, index_col=0)

In [5]:
b2019 = pd.read_csv('./data/Energy_and_Water_Data_2019.csv', low_memory=False, index_col=0)

In [6]:
b2018 = pd.read_csv('./data/Energy_and_Water_Data_2018.csv', low_memory=False, index_col=0)

In [7]:
b2018.shape, b2019.shape, b2020.shape, b2021.shape

((25245, 252), (29084, 253), (28067, 249), (29842, 248))

In [8]:
buildings.shape
# as we can see, there's a lot more entries in buildings, but way fewer features
# note: at the end of this notebook, we end up with 40 features again :D
# however, after cleaning the buildings df and removing columns with >50% nulls and repetitive columns,
# this was reduced to 22 features
# so the 40 in our final data set is an improvement. 

(59324, 40)

### Cleaning Functions

In [9]:
#function to make column names in snake case
def snake_case(dataframe):
    '''fn takes in a dataframe and changes all the column names to snake case'''
    dataframe.columns = [col.lower().replace(' ', '_') for col in dataframe.columns]
    return dataframe

In [10]:
#replace 'not found' or 'not available' with null values
def apply_nulls(dataframe):
    '''fn takes in a dataframe and changes 'Not found/not available/see bbl value' to Null'''
    dataframe = dataframe.replace('Not found',None)
    dataframe = dataframe.replace('Not Available',None)
    dataframe = dataframe.replace('',None)
    dataframe = dataframe.replace('See Primary BBL',None)
    dataframe = dataframe.replace('Insufficient Access',None)
    return dataframe

In [11]:
# convert object dtypes to numerics
def df_to_numeric(dataframe):
    '''fn takes in a dataframe and changes features to numeric types if possible'''
    for col in dataframe.columns:
        try:
            dataframe[col] = pd.to_numeric(dataframe[col])
        except:
            pass
    return dataframe

In [12]:
def inspect_columns(dataframe, col):
    '''fn for manually inspecting nulls and value counts in each column. 
    Useful to cross-reference with the data dictionary so that one can 
    determine what the values in the column mean'''
    print(f'Count unique values in "{col}" is {dataframe[col].nunique()}.')
    print(f'Count of nulls in "{col}" is {dataframe[col].isnull().sum()}.' if dataframe[col].isnull().sum() != 0 else 'No NULLs.')
    try:
        print(f'Mean is {round(np.nanmean(dataframe[col]), 2)}, median is {np.nanmedian(dataframe[col])}.')
    except Exception as e:
        print(f"Can't calculate statistics: {e}")
        pass
    print('Value Counts:')
    print(dataframe[col.strip()].value_counts())
    print('-------------------------------------------------------')

In [13]:
#what are all the feature names
buildings.columns

Index(['Property Name', 'Parent Property Id', 'Parent Property Name',
       'Street Number', 'Street Name', 'Borough',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type',
       'Largest Property Use Type - Gross Floor Area (ft²)',
       '2nd Largest Property Use Type',
       '2nd Largest Property Use - Gross Floor Area (ft²)',
       '3rd Largest Property Use Type',
       '3rd Largest Property Use Type - Gross Floor Area (ft²)', 'Year Built',
       'Occupancy', 'Metered Areas (Energy)', 'Metered Areas  (Water)',
       'ENERGY STAR Score', 'Source EUI (kBtu/ft²)',
       'Weather Normalized Source EUI (kBtu/ft²)', 'Site EUI (kBtu/ft²)',
       'Weather Normalized Site EUI (kBtu/ft²)',
       'Weather Normalized Site Electricity Intensity (kWh/ft²)',
       'Weather Normalized Site Natural Gas Intensity (therms/ft²)',
       'Fuel Oil #1 Use (kBtu)', 'Fuel Oil #2 Use (kBtu)',
       'Fuel Oil #4 U

In [14]:
buildings.dtypes

Property Name                                                  object
Parent Property Id                                             object
Parent Property Name                                           object
Street Number                                                  object
Street Name                                                    object
Borough                                                        object
Primary Property Type - Self Selected                          object
List of All Property Use Types at Property                     object
Largest Property Use Type                                      object
Largest Property Use Type - Gross Floor Area (ft²)             object
2nd Largest Property Use Type                                  object
2nd Largest Property Use - Gross Floor Area (ft²)              object
3rd Largest Property Use Type                                  object
3rd Largest Property Use Type - Gross Floor Area (ft²)         object
Year Built          

In [15]:
#number of nulls as a percentage
#definitely want to drop any features over 50%
#probably want to drop any features over 25% also, but need to investigate those features further
buildings.isnull().sum()/buildings.shape[0]*100.0

Property Name                                                  2.852134
Parent Property Id                                            57.858540
Parent Property Name                                          57.858540
Street Number                                                  5.001349
Street Name                                                    0.205650
Borough                                                        0.198908
Primary Property Type - Self Selected                          2.098645
List of All Property Use Types at Property                     2.120558
Largest Property Use Type                                      2.152586
Largest Property Use Type - Gross Floor Area (ft²)             2.912818
2nd Largest Property Use Type                                 47.828872
2nd Largest Property Use - Gross Floor Area (ft²)             47.828872
3rd Largest Property Use Type                                 55.879577
3rd Largest Property Use Type - Gross Floor Area (ft²)        55

In [16]:
buildings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59324 entries, 1 to 13223
Data columns (total 40 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Property Name                                               57632 non-null  object 
 1   Parent Property Id                                          25000 non-null  object 
 2   Parent Property Name                                        25000 non-null  object 
 3   Street Number                                               56357 non-null  object 
 4   Street Name                                                 59202 non-null  object 
 5   Borough                                                     59206 non-null  object 
 6   Primary Property Type - Self Selected                       58079 non-null  object 
 7   List of All Property Use Types at Property                  58066 non-null  object 
 

In [17]:
snake_case(buildings)

,property_name,parent_property_id,parent_property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),...,district_steam_use_(kbtu),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),water_use_(all_water_sources)_(kgal),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,NaN,NaN,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,...,1133475.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,3635.5,Not found
3,Cathedral Preparatory Seminary,NaN,NaN,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,...,NaN,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,102.9,Not found
4,The Nomad Hotel,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,...,NaN,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,10762.6,Not found
5,10 West 27 Street Corp,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,790.1,Not found
6,Westbury Realty,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,143.0,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13219,NaN,NaN,NaN,3010.0,VETERANS ROAD WEST,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13220,NaN,NaN,NaN,200.0,BOSCOMBE AVENUE,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13221,NaN,NaN,NaN,236.0,RICHMOND VALLEY ROAD,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation


In [18]:
buildings.iloc[0,5]

'Manhattan'

In [19]:
apply_nulls(buildings)

,property_name,parent_property_id,parent_property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),...,district_steam_use_(kbtu),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),water_use_(all_water_sources)_(kgal),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,NaN,NaN,None,None,Manhattan,Office,"Bank Branch, Office",Office,164754.0,...,1133475.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,3635.5,None
3,Cathedral Preparatory Seminary,NaN,NaN,None,None,Queens,K-12 School,K-12 School,K-12 School,94380.0,...,NaN,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,102.9,None
4,The Nomad Hotel,NaN,NaN,None,None,Manhattan,Hotel,Hotel,Hotel,125000.0,...,NaN,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,10762.6,None
5,10 West 27 Street Corp,NaN,NaN,None,None,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,790.1,None
6,Westbury Realty,NaN,NaN,None,None,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,143.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13219,NaN,NaN,NaN,3010.0,VETERANS ROAD WEST,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13220,NaN,NaN,NaN,200.0,BOSCOMBE AVENUE,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13221,NaN,NaN,NaN,236.0,RICHMOND VALLEY ROAD,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation


In [20]:
df = buildings.copy()
df

,property_name,parent_property_id,parent_property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),...,district_steam_use_(kbtu),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),water_use_(all_water_sources)_(kgal),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,NaN,NaN,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,...,1133475.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,3635.5,Not found
3,Cathedral Preparatory Seminary,NaN,NaN,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,...,NaN,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,102.9,Not found
4,The Nomad Hotel,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,...,NaN,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,10762.6,Not found
5,10 West 27 Street Corp,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,790.1,Not found
6,Westbury Realty,NaN,NaN,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,...,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,143.0,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13219,NaN,NaN,NaN,3010.0,VETERANS ROAD WEST,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13220,NaN,NaN,NaN,200.0,BOSCOMBE AVENUE,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13221,NaN,NaN,NaN,236.0,RICHMOND VALLEY ROAD,Staten Island,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation


In [21]:
df.dropna(axis=1, thresh=df.shape[0]*3/4, inplace=True)
df

,property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),year_built,occupancy,...,weather_normalized_site_electricity_intensity_(kwh/ft²),weather_normalized_site_natural_gas_intensity_(therms/ft²),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,1909,95.0,...,11.3,0.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,Not found
3,Cathedral Preparatory Seminary,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,1963,100.0,...,2.0,0.2,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,Not found
4,The Nomad Hotel,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,1999,85.0,...,18.8,0.7,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,Not found
5,10 West 27 Street Corp,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,1994,100.0,...,11.7,0.4,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,Not found
6,Westbury Realty,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,2012,0.0,...,6.0,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13219,NaN,3010.0,VETERANS ROAD WEST,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13220,NaN,200.0,BOSCOMBE AVENUE,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation
13221,NaN,236.0,RICHMOND VALLEY ROAD,Staten Island,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Violation


In [22]:
#drop any rows that have more than 75% nulls --> no point imputing data that's completely made up at that point
df.dropna(axis=0, thresh=df.shape[1]/4, inplace=True)

In [23]:
df.isnull().sum()/df.shape[0]*100.0

property_name                                                  0.769641
street_number                                                  4.614405
street_name                                                    0.210059
borough                                                        0.203172
primary_property_type_-_self_selected                          0.000000
list_of_all_property_use_types_at_property                     0.022383
largest_property_use_type                                      0.055097
largest_property_use_type_-_gross_floor_area_(ft²)             0.831626
year_built                                                     0.000000
occupancy                                                      0.769641
metered_areas_(energy)                                         4.554142
energy_star_score                                             19.532017
source_eui_(kbtu/ft²)                                          5.175709
weather_normalized_source_eui_(kbtu/ft²)                      13

In [24]:
df['total_ghg_emissions_(metric_tons_co2e)'].value_counts()

0.0              1238
0                 108
Not Available      74
299.8              26
384.5              23
                 ... 
2544.8              1
1460.2              1
1880.5              1
1790.4              1
5725.9              1
Name: total_ghg_emissions_(metric_tons_co2e), Length: 17147, dtype: int64

In [25]:
df_to_numeric(df)

,property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),year_built,occupancy,...,weather_normalized_site_electricity_intensity_(kwh/ft²),weather_normalized_site_natural_gas_intensity_(therms/ft²),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,1909,95.0,...,11.3,0.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,Not found
3,Cathedral Preparatory Seminary,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,1963,100.0,...,2.0,0.2,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,Not found
4,The Nomad Hotel,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,1999,85.0,...,18.8,0.7,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,Not found
5,10 West 27 Street Corp,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,1994,100.0,...,11.7,0.4,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,Not found
6,Westbury Realty,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,2012,0.0,...,6.0,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,St. Joseph by-the-Sea,5150.0,HYLAN BOULEVARD,Staten Island,K-12 School,K-12 School,K-12 School,184428.0,1963,100.0,...,4.3,0.5,8345200.1,85683.6,2734290.1,801374.5,670.0,443.3,226.8,In Compliance
11975,STATEN ISLAND UNIVERSITY HOSPITAL Seguine,375.0,SEGUINE AVENUE,Staten Island,Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),220240.0,1950,100.0,...,26.7,3.0,66815614.2,668156.1,20082283.2,5885779.8,5725.9,4060.3,1665.5,In Compliance
11976,N001597-S STATEN ISLAND NY,2790.0,ARTHUR KILL ROAD,Staten Island,Retail Store,Retail Store,Retail Store,116518.0,2012,100.0,...,16.6,0.1,1020290.0,9014.7,6655085.9,1931131.6,606.1,54.2,551.9,In Compliance


In [26]:
df.head()

,property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),year_built,occupancy,...,weather_normalized_site_electricity_intensity_(kwh/ft²),weather_normalized_site_natural_gas_intensity_(therms/ft²),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,1909,95.0,...,11.3,0.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,Not found
3,Cathedral Preparatory Seminary,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,1963,100.0,...,2.0,0.2,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,Not found
4,The Nomad Hotel,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,1999,85.0,...,18.8,0.7,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,Not found
5,10 West 27 Street Corp,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,1994,100.0,...,11.7,0.4,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,Not found
6,Westbury Realty,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,2012,0.0,...,6.0,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,Not found


In [27]:
df.dtypes

property_name                                                  object
street_number                                                  object
street_name                                                    object
borough                                                        object
primary_property_type_-_self_selected                          object
list_of_all_property_use_types_at_property                     object
largest_property_use_type                                      object
largest_property_use_type_-_gross_floor_area_(ft²)             object
year_built                                                     object
occupancy                                                     float64
metered_areas_(energy)                                         object
energy_star_score                                              object
source_eui_(kbtu/ft²)                                          object
weather_normalized_source_eui_(kbtu/ft²)                       object
site_eui_(kbtu/ft²) 

In [28]:
df['borough'].value_counts()
# lots of iterations of each borough - can be consolidated
# pine hill and patchogue are not in nyc (not in one of the 5 boroughs), so will drop them

Manhattan        23774
Brooklyn         12587
Bronx            10610
Queens            9539
Staten Island      959
Not Available      423
brooklyn            26
BROOKLYN            15
Brooklyn             9
bronx                8
QUEENS               3
staten Island        2
BRONX                2
Pine Hill            1
Patchogue            1
staten island        1
STATEN ISLAND        1
Name: borough, dtype: int64

In [29]:
#fix the borough values so that they all fall under the 5 official boroughs
#manhattan, bronx, brooklyn, queens, staten island

df['borough'] = df['borough'].str.replace('staten Island', 'Staten Island')
df['borough'] = df['borough'].str.replace('staten island', 'Staten Island')
df['borough'] = df['borough'].str.replace('STATEN ISLAND', 'Staten Island')

df['borough'] = df['borough'].str.replace('brooklyn', 'Brooklyn')
df['borough'] = df['borough'].str.replace('BROOKLYN', 'Brooklyn')
df['borough'] = df['borough'].str.replace('Brooklyn ', 'Brooklyn')


df['borough'] = df['borough'].str.replace('bronx', 'Bronx')
df['borough'] = df['borough'].str.replace('BRONX', 'Bronx')


df['borough'] = df['borough'].str.replace('QUEENS', 'Queens')

df['borough'] = df['borough'].str.replace('Manhattan ', 'Manhattan')

In [30]:
df.drop(index=df[df.borough == 'Pine Hill'].index, inplace=True)
df.drop(index=df[df.borough == 'Patchogue'].index, inplace=True)

In [31]:
df['borough'].value_counts()

Manhattan        23774
Brooklyn         12637
Bronx            10620
Queens            9541
Staten Island      963
Not Available      423
Name: borough, dtype: int64

In [32]:
df

,property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),year_built,occupancy,...,weather_normalized_site_electricity_intensity_(kwh/ft²),weather_normalized_site_natural_gas_intensity_(therms/ft²),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,1909,95.0,...,11.3,0.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,Not found
3,Cathedral Preparatory Seminary,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,1963,100.0,...,2.0,0.2,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,Not found
4,The Nomad Hotel,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,1999,85.0,...,18.8,0.7,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,Not found
5,10 West 27 Street Corp,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,1994,100.0,...,11.7,0.4,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,Not found
6,Westbury Realty,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,2012,0.0,...,6.0,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,St. Joseph by-the-Sea,5150.0,HYLAN BOULEVARD,Staten Island,K-12 School,K-12 School,K-12 School,184428.0,1963,100.0,...,4.3,0.5,8345200.1,85683.6,2734290.1,801374.5,670.0,443.3,226.8,In Compliance
11975,STATEN ISLAND UNIVERSITY HOSPITAL Seguine,375.0,SEGUINE AVENUE,Staten Island,Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),220240.0,1950,100.0,...,26.7,3.0,66815614.2,668156.1,20082283.2,5885779.8,5725.9,4060.3,1665.5,In Compliance
11976,N001597-S STATEN ISLAND NY,2790.0,ARTHUR KILL ROAD,Staten Island,Retail Store,Retail Store,Retail Store,116518.0,2012,100.0,...,16.6,0.1,1020290.0,9014.7,6655085.9,1931131.6,606.1,54.2,551.9,In Compliance


In [33]:
for each in df.year_built:
    try:
        each = each.astype('int64')
    except:
        pass

In [34]:
df[df.year_built == 'See Primary BBL']

,property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),year_built,occupancy,...,weather_normalized_site_electricity_intensity_(kwh/ft²),weather_normalized_site_natural_gas_intensity_(therms/ft²),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
331,NaN,NaN,Not Available,Manhattan,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance
332,NaN,NaN,Not Available,Manhattan,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance
336,NaN,NaN,Not Available,Manhattan,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance
337,NaN,NaN,Not Available,Manhattan,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not on CBL
338,NaN,NaN,Not Available,Manhattan,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11809,NaN,7600.0,SHORE FRONT PARKWAY,Queens,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance
11852,NaN,NaN,Not Available,Not Available,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance
11854,NaN,NaN,Not Available,Not Available,See Primary BBL,See Primary BBL,See Primary BBL,NaN,See Primary BBL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Compliance


In [35]:
df.year_built.unique()

array(['1909', '1963', '1999', '1994', '2012', '1925', '1975', '1950',
       '1977', '1969', '1931', '1868', '1930', '1905', '1936', '1923',
       '1954', '1998', '1940', '1991', '1979', '1953', '2009', '1972',
       '1928', '1973', '2011', '2005', '1980', '1904', '1955', '1926',
       '2015', '1968', '1961', '1899', '1952', '1981', '1951', '1989',
       '1944', '1937', '1959', '1960', '1912', '1922', '1938', '1935',
       '2007', '1939', '1915', '2006', '1962', '2002', '1920', '1927',
       '1965', '1929', '1971', '1946', '1900', '2008', '1921', '1984',
       '1976', '1987', '1913', '1982', '1910', '1916', '1983', '2014',
       '1958', '2003', '2004', '2017', '1967', '1956', '2001', '1911',
       '1924', '1902', '2016', '1908', '1985', '1881', '1974', '1917',
       '1896', '2013', '1964', '1992', '2000', '1901', '1897', '1957',
       '1888', '1996', '1990', '1907', '1970', '1918', '1890', '1914',
       '1941', '1894', '1943', '1903', '1898', '1948', '1895', '1880',
      

In [36]:
df

,property_name,street_number,street_name,borough,primary_property_type_-_self_selected,list_of_all_property_use_types_at_property,largest_property_use_type,largest_property_use_type_-_gross_floor_area_(ft²),year_built,occupancy,...,weather_normalized_site_electricity_intensity_(kwh/ft²),weather_normalized_site_natural_gas_intensity_(therms/ft²),natural_gas_use_(kbtu),weather_normalized_site_natural_gas_use_(therms),electricity_use_-_grid_purchase_(kbtu),weather_normalized_site_electricity_(kwh),total_ghg_emissions_(metric_tons_co2e),direct_ghg_emissions_(metric_tons_co2e),indirect_ghg_emissions_(metric_tons_co2e),dof_benchmarking_submission_status
Order,,,,,,,,,,,,,,,,,,,,,
1,The Argonaut Building,Not found,Not found,Manhattan,Office,"Bank Branch, Office",Office,164754.0,1909,95.0,...,11.3,0.1,1435754.7,16672.2,6551394.1,1920103.6,732.4,76.3,656.1,Not found
3,Cathedral Preparatory Seminary,Not found,Not found,Queens,K-12 School,K-12 School,K-12 School,94380.0,1963,100.0,...,2.0,0.2,2068300.1,23243.7,616343.7,184131.9,164.5,109.9,54.6,Not found
4,The Nomad Hotel,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,125000.0,1999,85.0,...,18.8,0.7,8245445.1,86776.9,8033914.4,2354605.3,1150.2,438.0,712.3,Not found
5,10 West 27 Street Corp,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,1994,100.0,...,11.7,0.4,1848519.4,20520.9,1976691.9,582516.1,273.4,98.2,175.3,Not found
6,Westbury Realty,Not found,Not found,Manhattan,Hotel,Hotel,Hotel,50000.0,2012,0.0,...,6.0,NaN,NaN,NaN,1022951.6,299809.9,90.7,0.0,90.7,Not found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,St. Joseph by-the-Sea,5150.0,HYLAN BOULEVARD,Staten Island,K-12 School,K-12 School,K-12 School,184428.0,1963,100.0,...,4.3,0.5,8345200.1,85683.6,2734290.1,801374.5,670.0,443.3,226.8,In Compliance
11975,STATEN ISLAND UNIVERSITY HOSPITAL Seguine,375.0,SEGUINE AVENUE,Staten Island,Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),Hospital (General Medical & Surgical),220240.0,1950,100.0,...,26.7,3.0,66815614.2,668156.1,20082283.2,5885779.8,5725.9,4060.3,1665.5,In Compliance
11976,N001597-S STATEN ISLAND NY,2790.0,ARTHUR KILL ROAD,Staten Island,Retail Store,Retail Store,Retail Store,116518.0,2012,100.0,...,16.6,0.1,1020290.0,9014.7,6655085.9,1931131.6,606.1,54.2,551.9,In Compliance


In [37]:
df_no_predict = df[df.energy_star_score.isnull()]
df_keep = df[~df.energy_star_score.isnull()]

In [38]:
df.shape, df_keep.shape, df_no_predict.shape

((58076, 26), (46733, 26), (11343, 26))

In [39]:
df_keep.isnull().sum()/44629

property_name                                                 0.000000
street_number                                                 0.044119
street_name                                                   0.002734
borough                                                       0.002644
primary_property_type_-_self_selected                         0.000000
list_of_all_property_use_types_at_property                    0.000000
largest_property_use_type                                     0.000000
largest_property_use_type_-_gross_floor_area_(ft²)            0.000000
year_built                                                    0.000000
occupancy                                                     0.000000
metered_areas_(energy)                                        0.034619
energy_star_score                                             0.000000
source_eui_(kbtu/ft²)                                         0.000000
weather_normalized_source_eui_(kbtu/ft²)                      0.090121
site_e

In [40]:
df_keep.dtypes

property_name                                                  object
street_number                                                  object
street_name                                                    object
borough                                                        object
primary_property_type_-_self_selected                          object
list_of_all_property_use_types_at_property                     object
largest_property_use_type                                      object
largest_property_use_type_-_gross_floor_area_(ft²)             object
year_built                                                     object
occupancy                                                     float64
metered_areas_(energy)                                         object
energy_star_score                                              object
source_eui_(kbtu/ft²)                                          object
weather_normalized_source_eui_(kbtu/ft²)                       object
site_eui_(kbtu/ft²) 

In [41]:
df_keep = df_keep.drop(columns=['borough', 'primary_property_type_-_self_selected', 
             'list_of_all_property_use_types_at_property', 'largest_property_use_type', 
             'largest_property_use_type_-_gross_floor_area_(ft²)', 'year_built', 'metered_areas_(energy)']).copy()

In [42]:
df_keep.dtypes

property_name                                                  object
street_number                                                  object
street_name                                                    object
occupancy                                                     float64
energy_star_score                                              object
source_eui_(kbtu/ft²)                                          object
weather_normalized_source_eui_(kbtu/ft²)                       object
site_eui_(kbtu/ft²)                                            object
weather_normalized_site_eui_(kbtu/ft²)                         object
weather_normalized_site_electricity_intensity_(kwh/ft²)        object
weather_normalized_site_natural_gas_intensity_(therms/ft²)     object
natural_gas_use_(kbtu)                                         object
weather_normalized_site_natural_gas_use_(therms)               object
electricity_use_-_grid_purchase_(kbtu)                         object
weather_normalized_s

In [43]:
#df_keep.to_csv('./data/modeling_data.csv', index = False)

---
### Combining newer data

In [44]:
b2018.shape, b2019.shape, b2020.shape, b2021.shape

((25245, 252), (29084, 253), (28067, 249), (29842, 248))

In [45]:
# the datasets all have slightly different feature numbers
# how many unique features are there in all four combined?
all_features = set(list(b2018.columns) + list(b2019.columns) + list(b2020.columns) + list(b2021.columns))
len(all_features)

303

In [46]:
# combine them by using concat
# and drop any duplicates (based on property name) since yearly data is going to have lots of repeats
# apply merge with newest data first so the newer data is kept
combined = pd.concat([b2021,b2020,b2019,b2018]).drop_duplicates(subset=['Property Name']).reset_index(drop=True)
combined

,Property Name,Parent Property Id,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,City,Postal Code,Primary Property Type - Self Selected,...,Hotel - Percent That Can Be Cooled,Estimated Data Flag - Municipally Supplied Potable Water – Outdoor Use,Data Quality Checker Run?,Water Current Date,Water Score (Multifamily Only),Irrigated Area (ft²),"Supermarket/Grocery - Open or Closed Refrigeration Density (Number per 1,000 ft²)",Convenience Store With Gas Station - Number of Open or Closed Refrigeration/Freezer Units,"Convenience Store With Gas Station - Open or Closed Refrigeration Density (Number per 1,000 ft²)","Senior Care Community - Living Unit Density (Number per 1,000 ft²)"
0,Astoria Blvd Property,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,4006520042,4538607,36-16 Astoria Blvd.,Long Island City,11103,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stellar - 70 West 93rd Street,20599688,Stellar - Campus West 93rd Street,12/31/2021,1-01206-0001,1031627,70 West 93rd Street,New York,10025,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Stellar - 28-50 West 97th Street,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,1-01832-0043,1055265,28-50 West 97th Street,New York,10025,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stellar - 310 Greenwich St,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,1-00142-0025,1083157;1083235;1083237;1083240;1083241;108324...,310 Greenwich Street / 40 Harrison Street,New York,10013,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stellar - 87 Hamilton Place,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,1-02072-0039,1061772,87 Hamilton Place,New York,10031,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41390,4 EAST 88 STREET,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2018,1014990065,1046924,4 EAST 88 STREET,NEW YORK,NaN,Multifamily Housing,...,Not Available,Not Available,Yes,Not Available,Not Available,0,Not Available,Not Available,Not Available,Not Available
41391,Ashley Young LLC,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2018,4007230008,4456928;4436930;4436932,47-15 34th Ave,Long Island City,NaN,Retail Store,...,Not Available,Not Available,Yes,Not Available,Not Available,0,Not Available,Not Available,Not Available,Not Available
41392,175 WEST 137 STREET,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2018,1020060007,1081808,175 WEST 137 STREET,NEW YORK,NaN,Office,...,Not Available,Not Available,No,06/30/2019,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
41393,Child1_Manhattan Beach,2670640,Parent_Manhattan Beach,12/31/2018,3871927,3245695,156 West End Avenue,Brooklyn,NaN,Multifamily Housing,...,Not Available,Not Available,No,Not Available,Not Available,0,Not Available,Not Available,Not Available,Not Available


In [47]:
# double check that the majority of data is from 2021 
# the other values would be properties which weren't listed in newer portfolio manager data for some reason
# this isn't a time series project, so I will be dropping these years later on
combined['Year Ending'].value_counts()

12/31/2021    29449
12/31/2020     4498
12/31/2019     4178
12/31/2018     3270
Name: Year Ending, dtype: int64

---
### Cleaning Newer Data

In [48]:
#apply snake case
snake_case(combined).head()

,property_name,parent_property_id,parent_property_name,year_ending,"nyc_borough,_block_and_lot_(bbl)",nyc_building_identification_number_(bin),address_1,city,postal_code,primary_property_type_-_self_selected,...,hotel_-_percent_that_can_be_cooled,estimated_data_flag_-__municipally_supplied_potable_water_–_outdoor_use,data_quality_checker_run?,water_current_date,water_score_(multifamily_only),irrigated_area_(ft²),"supermarket/grocery_-_open_or_closed_refrigeration_density_(number_per_1,000_ft²)",convenience_store_with_gas_station_-_number_of_open_or_closed_refrigeration/freezer_units,"convenience_store_with_gas_station_-_open_or_closed_refrigeration_density_(number_per_1,000_ft²)","senior_care_community_-_living_unit_density_(number_per_1,000_ft²)"
0,Astoria Blvd Property,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,4006520042,4538607,36-16 Astoria Blvd.,Long Island City,11103,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Stellar - 70 West 93rd Street,20599688,Stellar - Campus West 93rd Street,12/31/2021,1-01206-0001,1031627,70 West 93rd Street,New York,10025,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Stellar - 28-50 West 97th Street,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,1-01832-0043,1055265,28-50 West 97th Street,New York,10025,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stellar - 310 Greenwich St,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,1-00142-0025,1083157;1083235;1083237;1083240;1083241;108324...,310 Greenwich Street / 40 Harrison Street,New York,10013,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stellar - 87 Hamilton Place,Not Applicable: Standalone Property,Not Applicable: Standalone Property,12/31/2021,1-02072-0039,1061772,87 Hamilton Place,New York,10031,Multifamily Housing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# are any Energy Scores null?
combined['energy_star_score'].isnull().sum()
#looks like no nulls

0

In [50]:
combined['energy_star_score'].value_counts()
#but there are 9909 Not Available values, which are nulls

Not Available    9909
100              2267
1                 888
99                452
78                410
                 ... 
23                187
24                186
15                184
22                173
13                169
Name: energy_star_score, Length: 101, dtype: int64

In [51]:
# apply_nulls(combined)

In [52]:
# apply_nulls function didn't change anything for some reason, 
# so here are the same steps manually
combined = combined.replace('Not found',None)
combined = combined.replace('Not Available',None)
combined = combined.replace('',None)
combined = combined.replace('See Primary BBL',None)
combined = combined.replace('Insufficient access',None)

In [53]:
# now there are no 'Not Available's (they've all been converted to nulls)
combined['energy_star_score'].value_counts()

100    2267
1       888
99      452
78      410
86      407
       ... 
23      187
24      186
15      184
22      173
13      169
Name: energy_star_score, Length: 100, dtype: int64

In [54]:
combined['energy_star_score'].isnull().sum()

9909

In [55]:
# drop all observations where energy star score is empty 
# because I can't make any predictions off of these
# if I have a decent model at the end, then I could calculate scores for these (but can't verify)
combined = combined[~combined.energy_star_score.isnull()]
combined.shape

(31486, 303)

In [56]:
# Number of nulls as a percentage
# Definitely want to drop any features over 50% null
# Probably want to drop any features over 25% also but # need to investigate those features further

combined.isnull().mean().mul(100).sort_values(ascending=False)

convenience_store_with_gas_station_-_open_or_closed_refrigeration_density_(number_per_1,000_ft²)    100.0
convenience_store_with_gas_station_-_number_of_open_or_closed_refrigeration/freezer_units           100.0
reason(s)_for_no_score                                                                              100.0
barracks-_gross_floor_area_(ft²)                                                                    100.0
kerosene_use_(kbtu)                                                                                 100.0
                                                                                                    ...  
net_emissions_(metric_tons_co2e)                                                                      0.0
source_energy_use_(kbtu)                                                                              0.0
national_median_source_eui_(kbtu/ft²)                                                                 0.0
source_eui_(kbtu/ft²)                         

In [57]:
# Columns with missing value percentage of 50% or more:
combined.columns[(combined.isnull().mean().mul(100) > 50.0)]

Index(['2nd_largest_property_use_type',
       '2nd_largest_property_use_-_gross_floor_area_(ft²)',
       '3rd_largest_property_use_type',
       '3rd_largest_property_use_type_-_gross_floor_area_(ft²)',
       'target_energy_star_score', 'reason(s)_for_no_score',
       'energy_star_certification_-_year(s)_certified_(score)',
       'energy_star_certification_-_last_approval_date',
       'fuel_oil_#1_use_(kbtu)', 'fuel_oil_#2_use_(kbtu)',
       ...
       'hotel_-_percent_that_can_be_cooled',
       'estimated_data_flag_-__municipally_supplied_potable_water_–_outdoor_use',
       'data_quality_checker_run?', 'water_current_date',
       'water_score_(multifamily_only)', 'irrigated_area_(ft²)',
       'supermarket/grocery_-_open_or_closed_refrigeration_density_(number_per_1,000_ft²)',
       'convenience_store_with_gas_station_-_number_of_open_or_closed_refrigeration/freezer_units',
       'convenience_store_with_gas_station_-_open_or_closed_refrigeration_density_(number_per_1,000_f

In [58]:
# Columns with missing value percentage between 25% and 50%:
combined.columns[(combined.isnull().mean().mul(100).between(25.0, 49.99))]

Index(['postal_code', 'metered_areas_(water)',
       'multifamily_housing_-_gross_floor_area_(ft²)',
       'multifamily_housing_-_number_of_bedrooms',
       'multifamily_housing_-_number_of_bedrooms_density_(number_per_1,000_sq_ft)',
       'multifamily_housing_-_number_of_residential_living_units_in_a_high-rise_building_(10_or_more_stories)',
       'multifamily_housing_-_total_number_of_residential_living_units',
       'multifamily_housing_-_total_number_of_residential_living_units_density_(number_per_1,000_sq_ft)',
       'multifamily_housing_-_number_of_residential_living_units_in_a_high-rise_building_density_(number_per_1,000_sq_ft)',
       'multifamily_housing_-_number_of_residential_living_units_in_a_mid-rise_building_(5-9_stories)',
       'multifamily_housing_-_number_of_residential_living_units_in_a_mid-rise_building_density_(number_per_1,000_sq_ft)',
       'multifamily_housing_-_percent_that_can_be_cooled',
       'multifamily_housing_-_percent_that_can_be_heated',
   

In [59]:
# drop any columns with more than 50% nulls
# will analyze other columns individually and see if they're worth keeping or not
combined = combined.dropna(axis=1, thresh=combined.shape[0]/2).copy()

In [60]:
# here's an example of the output for 5 features, 
# but I went through this manually for each one that was left after dropping 50% nulls
for column in list(combined.columns[25:27]) + list(combined.columns[67:70]):
    inspect_columns(combined, column)
    print()

Count unique values in "weather_normalized_site_eui_(kbtu/ft²)" is 2421.
Count of nulls in "weather_normalized_site_eui_(kbtu/ft²)" is 2386.
Can't calculate statistics: can only concatenate str (not "NoneType") to str
Value Counts:
85.5     53
81.5     51
82.6     51
83.5     51
83.4     51
         ..
286.4     1
402.5     1
255.4     1
187.9     1
216       1
Name: weather_normalized_site_eui_(kbtu/ft²), Length: 2421, dtype: int64
-------------------------------------------------------

Count unique values in "national_median_site_eui_(kbtu/ft²)" is 1800.
No NULLs.
Can't calculate statistics: 'str' object has no attribute 'dtype'
Value Counts:
86       95
89       88
88.4     88
82.3     87
89.5     86
         ..
127.5     1
256.4     1
145.1     1
187.1     1
21.9      1
Name: national_median_site_eui_(kbtu/ft²), Length: 1800, dtype: int64
-------------------------------------------------------

Count unique values in "multifamily_housing_-_percent_that_can_be_cooled" is 38.
Count 

My thoughts:
* In these 5 example features, I would keep the first two since there's less than 10% nulls and no values that stand out as red flags.
* I would drop the next two (`multifamily_housing_-_percent_that_can_be_cooled/heated`) for a couple reasons:
    * lots of null values
    * lots of '0' values (not the most, but still a lot
    * based on the data dictionary, this is an example of a sub-feature.  All the various property types (there are 22 total) had a bunch of their own columns with values related to size/number of rooms/energy used.  All of the other property type sub-features had over 50% nulls but because so many units were multi-family housing, these are still in my list. It doesn't make sense to only keep these as features if all the other property type sub-features are gone, so I will remove all of these as well
* I would drop the last feature as well because the data is heavily weighted towards one value and it doesn't tell me anything useful about possible energy star score

In [61]:
# there's 108 columns left after dropping >50% nulls
# I went through all of them one by one
print(combined.shape)
combined.columns

(31486, 108)


Index(['property_name', 'parent_property_id', 'parent_property_name',
       'year_ending', 'nyc_borough,_block_and_lot_(bbl)',
       'nyc_building_identification_number_(bin)', 'address_1', 'city',
       'postal_code', 'primary_property_type_-_self_selected',
       ...
       'number_of_active_it_meters', 'generation_date', 'submission_date',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'nta', 'borough'],
      dtype='object', length=108)

In [62]:
# this is the result of going through each of the columns one-by-one
col_to_drop = ['parent_property_id', 'parent_property_name',      # unique identifiers, equivalent to index
               'year_ending',       # not relevant since this is not a timeseries project
               'nyc_borough,_block_and_lot_(bbl)', 'nyc_building_identification_number_(bin)',       # unique identifiers, equivalent to index
               'address_1', 'city', 'postal_code',       # unique identifiers, equivalent to index; also too identifying
               'primary_property_type_-_self_selected',       # only keeping primary_property_type_-_portfolio_manager-calculated
               'national_median_reference_property_type',       # only keeping primary_property_type_-_portfolio_manager-calculated
               'list_of_all_property_use_types_at_property',       # only keeping primary_property_type_-_portfolio_manager-calculated
               'largest_property_use_type',       # only keeping primary_property_type_-_portfolio_manager-calculated
               'largest_property_use_type_-_gross_floor_area_(ft²)',       # only keeping primary_property_type_-_portfolio_manager-calculated
               'construction_status',      # mostly 'existing', not useful
               'metered_areas_(energy)', 'metered_areas_(water)',       # just 'whole building' vs 'whole property'. not informative
               'national_median_energy_star_score',       # 50.0 for all observations. this is a benchmark, not useful as a feature
               'energy_star_certification_-_eligibility',       # all of these observations have scores, so this is not relevant
               'green_power_-_offsite_(kwh)', 'green_power_-_onsite_and_offsite_(kwh)',       # too many nulls and '0' values
               'avoided_emissions_-_offsite_green_power_(metric_tons_co2e)',       # too many nulls and '0' values
               'avoided_emissions_-_onsite_and_offsite_green_power_(metric_tons_co2e)',       # too many nulls and '0' values
               'multifamily_housing_-_gross_floor_area_(ft²)',       # removing all property_type sub-features
               'multifamily_housing_-_number_of_bedrooms',       # removing all property_type sub-features
               'multifamily_housing_-_number_of_bedrooms_density_(number_per_1,000_sq_ft)',       # removing all property_type sub-features
               'multifamily_housing_-_total_number_of_residential_living_units',       # removing all property_type sub-features
               'multifamily_housing_-_total_number_of_residential_living_units_density_(number_per_1,000_sq_ft)',       # removing all property_type sub-features
               'multifamily_housing_-_number_of_residential_living_units_in_a_high-rise_building_(10_or_more_stories)',       # removing all property_type sub-features
               'multifamily_housing_-_number_of_residential_living_units_in_a_high-rise_building_density_(number_per_1,000_sq_ft)',       # removing all property_type sub-features
               'multifamily_housing_-_number_of_residential_living_units_in_a_mid-rise_building_(5-9_stories)',       # removing all property_type sub-features
               'multifamily_housing_-_number_of_residential_living_units_in_a_mid-rise_building_density_(number_per_1,000_sq_ft)',       # removing all property_type sub-features
               'multifamily_housing_-_percent_that_can_be_cooled',       # removing all property_type sub-features
               'multifamily_housing_-_percent_that_can_be_heated',       # removing all property_type sub-features
               'estimated_data_flag_-_electricity_(grid_purchase)', 'estimated_data_flag_-_natural_gas',        # too many nulls and '0' values
               'estimated_values_-_energy', 'estimated_values_-_water',       # too many nulls and '0' values
               'alert_-_data_center_issue_(with_estimates,_it_configuration,_or_it_meter)',       # too many nulls and '0' values
               'alert_-_energy_meter_has_less_than_12_full_calendar_months_of_data',       # too many nulls and '0' values
               'alert_-_energy_meter_has_gaps',        # too many nulls and '0' values
               'alert_-_energy_meter_has_overlaps',       # too many nulls and '0' values
               'alert_-_energy_-_no_meters_selected_for_metrics',        # too many nulls and '0' values
               'alert_-_energy_meter_has_single_entry_more_than_65_days',       # too many nulls and '0' values
               'alert_-_water_meter_has_less_than_12_full_calendar_months_of_data',       # too many nulls and '0' values
               'alert_-_property_has_no_uses',        # too many nulls and '0' values
               'default_values', 'temporary_values',       # too many nulls and '0' values
               'property_gfa_-_calculated_(buildings_and_parking)_(ft²)',       # only keeping property_gfa_-_self-reported_(ft²)
               'property_gfa_-_calculated_(buildings)_(ft²)',        # only keeping property_gfa_-_self-reported_(ft²)
               'indoor_water_use_(all_water_sources)_(kgal)',       # too many nulls and '0' values
               'municipally_supplied_potable_water_-_indoor_use_(kgal)',       # too many nulls and '0' values
               'municipally_supplied_potable_water_-_total_use_(all_meter_types)_(kgal)',       # too many nulls and '0' values
               'last_modified_date_-_property',        # too many nulls and the majority of values appeared to be date of data collection, so not helpful
               'last_modified_date_-_electric_meters',       # too many nulls and the majority of values appeared to be date of data collection, so not helpful
               'last_modified_date_-_gas_meters',        # too many nulls and the majority of values appeared to be date of data collection, so not helpful
               'last_modified_date_-_water_meters',        # too many nulls and the majority of values appeared to be date of data collection, so not helpful
               'last_modified_date_-_property_use',        # too many nulls and the majority of values appeared to be date of data collection, so not helpful
               'number_of_active_energy_meters_-_used_to_compute_metrics',      # only keeping number_of_active_energy_meters_-_total
               'number_of_active_energy_meters_-_not_used_to_compute_metrics',       # only keeping number_of_active_energy_meters_-_total
               'number_of_active_it_meters',       # only keeping number_of_active_energy_meters_-_total
               'generation_date', 'submission_date',       # unique identifiers, equivalent to index
               'latitude', 'longitude',       # unique identifiers, equivalent to index
               'community_board', 'council_district', 'census_tract', 'nta']      # too many nulls and 'Not Available' values


In [63]:
# create new dataframe that drops all the features listed above
model_data = combined.drop(columns=col_to_drop).copy()
model_data

,property_name,primary_property_type_-_portfolio_manager-calculated,year_built,number_of_buildings,occupancy,energy_star_score,site_eui_(kbtu/ft²),weather_normalized_site_eui_(kbtu/ft²),national_median_site_eui_(kbtu/ft²),site_energy_use_(kbtu),...,indirect_ghg_emissions_(metric_tons_co2e),net_emissions_(metric_tons_co2e),indirect_ghg_emissions_intensity_(kgco2e/ft²),national_median_total_ghg_emissions_(metric_tons_co2e),egrid_output_emissions_rate_(kgco2e/mbtu),percent_of_electricity_that_is_green_power,property_gfa_-_self-reported_(ft²),water_use_(all_water_sources)_(kgal),number_of_active_energy_meters_-_total,borough
0,Astoria Blvd Property,Multifamily Housing,2010,1,100,71,66.7,70.4,76.1,1666390.8,...,51.1,107.5,2,122.7,84.5,0,25000.0,910.8,8.0,QUEENS
1,Stellar - 70 West 93rd Street,Multifamily Housing,1970,1,100,100,19.2,19.2,45.5,5022480.9,...,354.7,398.6,1.4,945.9,84.5,0,261980.0,12826.8,6.0,MANHATTAN
2,Stellar - 28-50 West 97th Street,Multifamily Housing,1943,1,100,84,66.9,72,87.1,22315905.8,...,370.3,1323,1.1,1721.6,84.5,0,333342.0,16038,3.0,MANHATTAN
3,Stellar - 310 Greenwich St,Multifamily Housing,1975,3,100,3,113.8,121.1,70.5,126690147.9,...,2249.2,7582.1,2,4697.7,84.5,0,1112900.0,68151.7,4.0,NaN
4,Stellar - 87 Hamilton Place,Multifamily Housing,1941,1,100,24,114.6,None,96,8329013.8,...,110.3,483.4,1.5,404.8,84.5,0,72662.0,7450.3,11.0,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41386,336 WEST END AVENUE,Multifamily Housing,1931,1,100,7,143,142.6,99.9,18130211.4,...,151.8,1358.5,NaN,949,84.7,0,126811.0,None,NaN,MANHATTAN
41388,1681 2 AVENUE,Multifamily Housing,1966,1,100,59,79.4,78.6,84.6,13451032.4,...,276.1,817.4,NaN,871.6,84.7,0,169507.0,None,NaN,MANHATTAN
41389,460 EAST 79 STREET,Multifamily Housing,1957,1,100,66,74.7,75,83.7,11741340.2,...,267.8,723.5,NaN,810.7,84.7,0,157260.0,7442.9,NaN,MANHATTAN
41390,4 EAST 88 STREET,Multifamily Housing,1922,1,100,47,93.7,94.6,92.2,5408025.1,...,91.6,321.4,NaN,316.2,84.7,0,57726.0,None,NaN,MANHATTAN


In [64]:
# try to convert each column from strings to numerics
df_to_numeric(model_data)

,property_name,primary_property_type_-_portfolio_manager-calculated,year_built,number_of_buildings,occupancy,energy_star_score,site_eui_(kbtu/ft²),weather_normalized_site_eui_(kbtu/ft²),national_median_site_eui_(kbtu/ft²),site_energy_use_(kbtu),...,indirect_ghg_emissions_(metric_tons_co2e),net_emissions_(metric_tons_co2e),indirect_ghg_emissions_intensity_(kgco2e/ft²),national_median_total_ghg_emissions_(metric_tons_co2e),egrid_output_emissions_rate_(kgco2e/mbtu),percent_of_electricity_that_is_green_power,property_gfa_-_self-reported_(ft²),water_use_(all_water_sources)_(kgal),number_of_active_energy_meters_-_total,borough
0,Astoria Blvd Property,Multifamily Housing,2010,1,100,71,66.7,70.4,76.1,1666390.8,...,51.1,107.5,2.0,122.7,84.5,0.0,25000.0,910.8,8.0,QUEENS
1,Stellar - 70 West 93rd Street,Multifamily Housing,1970,1,100,100,19.2,19.2,45.5,5022480.9,...,354.7,398.6,1.4,945.9,84.5,0.0,261980.0,12826.8,6.0,MANHATTAN
2,Stellar - 28-50 West 97th Street,Multifamily Housing,1943,1,100,84,66.9,72.0,87.1,22315905.8,...,370.3,1323.0,1.1,1721.6,84.5,0.0,333342.0,16038.0,3.0,MANHATTAN
3,Stellar - 310 Greenwich St,Multifamily Housing,1975,3,100,3,113.8,121.1,70.5,126690147.9,...,2249.2,7582.1,2.0,4697.7,84.5,0.0,1112900.0,68151.7,4.0,NaN
4,Stellar - 87 Hamilton Place,Multifamily Housing,1941,1,100,24,114.6,NaN,96.0,8329013.8,...,110.3,483.4,1.5,404.8,84.5,0.0,72662.0,7450.3,11.0,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41386,336 WEST END AVENUE,Multifamily Housing,1931,1,100,7,143.0,142.6,99.9,18130211.4,...,151.8,1358.5,NaN,949.0,84.7,0.0,126811.0,NaN,NaN,MANHATTAN
41388,1681 2 AVENUE,Multifamily Housing,1966,1,100,59,79.4,78.6,84.6,13451032.4,...,276.1,817.4,NaN,871.6,84.7,0.0,169507.0,NaN,NaN,MANHATTAN
41389,460 EAST 79 STREET,Multifamily Housing,1957,1,100,66,74.7,75.0,83.7,11741340.2,...,267.8,723.5,NaN,810.7,84.7,0.0,157260.0,7442.9,NaN,MANHATTAN
41390,4 EAST 88 STREET,Multifamily Housing,1922,1,100,47,93.7,94.6,92.2,5408025.1,...,91.6,321.4,NaN,316.2,84.7,0.0,57726.0,NaN,NaN,MANHATTAN


In [65]:
# make sure all my data types look correct and model-able
model_data.dtypes
# property_name will not be in features
# can dummify primary_property_type/borough
# should check what values they cover

property_name                                                                          object
primary_property_type_-_portfolio_manager-calculated                                   object
year_built                                                                              int64
number_of_buildings                                                                     int64
occupancy                                                                               int64
energy_star_score                                                                       int64
site_eui_(kbtu/ft²)                                                                   float64
weather_normalized_site_eui_(kbtu/ft²)                                                float64
national_median_site_eui_(kbtu/ft²)                                                   float64
site_energy_use_(kbtu)                                                                float64
weather_normalized_site_energy_use_(kbtu)                   

In [66]:
# primary_property_type_-_portfolio_manager-calculated                                                object
# borough                                                                                             object

# analyze value counts
print(model_data['primary_property_type_-_portfolio_manager-calculated'].value_counts(normalize=True))
print(model_data['borough'].nunique())

#definitely can make dummies for primary property and borough

Multifamily Housing                      0.739122
Office                                   0.092454
K-12 School                              0.068665
Hotel                                    0.025376
Non-Refrigerated Warehouse               0.018834
Retail Store                             0.009369
Worship Facility                         0.007495
Residence Hall/Dormitory                 0.006638
Distribution Center                      0.006606
Hospital (General Medical & Surgical)    0.004605
Senior Living Community                  0.004288
Medical Office                           0.004065
Supermarket/Grocery Store                0.003462
Senior Care Community                    0.002033
Courthouse                               0.001620
Refrigerated Warehouse                   0.001556
Financial Office                         0.001048
Wastewater Treatment Plant               0.000985
Mixed Use Property                       0.000826
Wholesale Club/Supercenter               0.000508


---
### Saving cleaned data

In [67]:
# save cleaned combined data to file for EDA
combined.to_csv('./data/combined_data.csv', index=False)

In [68]:
# save cleaned model data to file for modeling
model_data.to_csv('./data/modeling_data.csv', index=False)